# Convolutional Neural Networks and Computer Vision

In [33]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt

In [34]:
dir_path = '10_food_classes_all_data'
train_path = os.path.join(dir_path,'train')
test_path = os.path.join(dir_path,'test')

In [35]:
for dirpath, dirnames, filenames in os.walk(dir_path):
    if len(dirnames) != 0 :
        print()
    print(f'{len(dirnames)} dir and {len(filenames)} images >> "{dirpath}"')


2 dir and 0 images >> "10_food_classes_all_data"

10 dir and 0 images >> "10_food_classes_all_data\test"
0 dir and 250 images >> "10_food_classes_all_data\test\chicken_curry"
0 dir and 250 images >> "10_food_classes_all_data\test\chicken_wings"
0 dir and 250 images >> "10_food_classes_all_data\test\fried_rice"
0 dir and 250 images >> "10_food_classes_all_data\test\grilled_salmon"
0 dir and 250 images >> "10_food_classes_all_data\test\hamburger"
0 dir and 250 images >> "10_food_classes_all_data\test\ice_cream"
0 dir and 250 images >> "10_food_classes_all_data\test\pizza"
0 dir and 250 images >> "10_food_classes_all_data\test\ramen"
0 dir and 250 images >> "10_food_classes_all_data\test\steak"
0 dir and 250 images >> "10_food_classes_all_data\test\sushi"

10 dir and 0 images >> "10_food_classes_all_data\train"
0 dir and 750 images >> "10_food_classes_all_data\train\chicken_curry"
0 dir and 750 images >> "10_food_classes_all_data\train\chicken_wings"
0 dir and 750 images >> "10_food_clas

In [45]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(42)

#preprocess data (and scaling)
train_datagen = ImageDataGenerator(rescale=1/255., rotation_range=.2,shear_range=.2,
                                    zoom_range=.2,width_shift_range=.2,
                                    height_shift_range=.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1/255.)

#import data and turn to batches
train_data = train_datagen.flow_from_directory(directory=train_path,
                                                batch_size=32,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                seed=42)

test_data = test_datagen.flow_from_directory(directory=test_path,
                                                batch_size=32,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                seed=42,shuffle=False)

Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [46]:
class_names = list(train_data.class_indices.keys())
class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [42]:
tf.random.set_seed(42)
model_1 = tf.keras.Sequential([ 
    tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='valid'),

    tf.keras.layers.Conv2D(filters= 10,kernel_size= (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='valid'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

# because in datagen return onehot in 'categorical' mode
model_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), 
                optimizer=tf.keras.optimizers.Adam(), 
                metrics=['accuracy'])

history_1 = model_1.fit(train_data, epochs=5, 
                        steps_per_epoch=len(train_data),
                        validation_data=test_data, 
                        validation_steps=len(test_data))

Epoch 1/5
235/235 [==============================] - 166s 707ms/step - loss: 2.1804 - accuracy: 0.2093 - val_loss: 1.9523 - val_accuracy: 0.3268
Epoch 2/5
235/235 [==============================] - 140s 596ms/step - loss: 2.0494 - accuracy: 0.2733 - val_loss: 1.8537 - val_accuracy: 0.3536
Epoch 3/5
235/235 [==============================] - 137s 583ms/step - loss: 1.9850 - accuracy: 0.3093 - val_loss: 1.8066 - val_accuracy: 0.3792
Epoch 4/5
235/235 [==============================] - 138s 586ms/step - loss: 1.9324 - accuracy: 0.3308 - val_loss: 1.7999 - val_accuracy: 0.3772
Epoch 5/5
235/235 [==============================] - 138s 585ms/step - loss: 1.9148 - accuracy: 0.3345 - val_loss: 1.7391 - val_accuracy: 0.4116


In [56]:
model_1.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 222, 222, 10)      280       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 111, 111, 10)     0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 109, 109, 10)      910       
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 54, 54, 10)       0         
 g2D)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 29160)             0         
                                                                 
 dense_8 (Dense)             (None, 10)               

### Apply from some experience from model_1 

In [61]:
for layer in model_1.layers[:-2] :
    layer.trainable = False

In [73]:
model_2 = tf.keras.Sequential([ 
    model_1.layers[0],
    model_1.layers[1],
    model_1.layers[2],
    model_1.layers[3],

    tf.keras.layers.Conv2D(filters= 10,kernel_size= (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model_2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), 
                optimizer=tf.keras.optimizers.Adam(), 
                metrics=['accuracy'])

history_2 = model_2.fit(train_data, epochs=5, 
                        steps_per_epoch=len(train_data),
                        validation_data=test_data, 
                        validation_steps=len(test_data))

Epoch 1/5
235/235 [==============================] - 99s 421ms/step - loss: 2.0984 - accuracy: 0.2476 - val_loss: 1.8264 - val_accuracy: 0.3704
Epoch 2/5
235/235 [==============================] - 99s 421ms/step - loss: 1.9207 - accuracy: 0.3357 - val_loss: 1.7928 - val_accuracy: 0.3864
Epoch 3/5
235/235 [==============================] - 99s 421ms/step - loss: 1.8843 - accuracy: 0.3495 - val_loss: 1.7818 - val_accuracy: 0.3764
Epoch 4/5
235/235 [==============================] - 98s 418ms/step - loss: 1.8589 - accuracy: 0.3564 - val_loss: 1.6873 - val_accuracy: 0.4272
Epoch 5/5
235/235 [==============================] - 106s 452ms/step - loss: 1.8441 - accuracy: 0.3688 - val_loss: 1.6756 - val_accuracy: 0.4312


In [75]:
model_2.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 222, 222, 10)      280       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 111, 111, 10)     0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 109, 109, 10)      910       
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 54, 54, 10)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 52, 52, 10)        910       
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 26, 26, 10)     

In [74]:
print(model_1.evaluate(test_data))
print(model_2.evaluate(test_data))

79/79 [==============================] - 9s 115ms/step - loss: 1.7391 - accuracy: 0.4116
[1.7391011714935303, 0.4115999937057495]
79/79 [==============================] - 9s 116ms/step - loss: 1.6756 - accuracy: 0.4312
[1.6755839586257935, 0.4311999976634979]


> slightly improve accuracy.